In [1]:
import ast
import os

import pandas as pd

our_dataset_path = '.'

posts_path = os.path.join(our_dataset_path, 'Data\\posts.csv')
fact_checks_path = os.path.join(our_dataset_path, 'Data\\fact_checks.csv')
fact_check_post_mapping_path = os.path.join(our_dataset_path, 'Data\\pairs.csv')

for path in [posts_path, fact_checks_path, fact_check_post_mapping_path]:
    print(f"Checking path: {path}")
    assert os.path.isfile(path), f"File not found: {path}"


Checking path: .\Data\posts.csv
Checking path: .\Data\fact_checks.csv
Checking path: .\Data\pairs.csv


In [ ]:
#This is purely to sort the CVs numbers and not otherwise relevant to the main code - please disable for the code to work as intended
#import numpy as np

#df = pd.read_csv('Data\\pairs.csv', header=None)
#df.columns = ['post_id', 'fact_check_id']
#df.sort_values('post_id', ascending=False, inplace=True, key=lambda s: s.groupby(s).transform("size"))
#print (df.sort_values)

<bound method DataFrame.sort_values of       post_id fact_check_id
343      5767         19000
352      5767        143651
344      5767         19801
345      5767         98205
346      5767        140676
...       ...           ...
10955    2321         62043
10954     699         62043
10953   24032         62038
10952   10828         62028
25743   25831        202891

[25744 rows x 2 columns]>


In [2]:
parse_col = lambda s: ast.literal_eval(s.replace('\n', '\\n')) if s else s

df_fact_checks = pd.read_csv(fact_checks_path).fillna('').set_index('fact_check_id')
for col in ['claim', 'instances', 'title']:
    df_fact_checks[col] = df_fact_checks[col].apply(parse_col)


df_posts = pd.read_csv(posts_path).fillna('').set_index('post_id')
for col in ['instances', 'ocr', 'verdicts', 'text']:
    df_posts[col] = df_posts[col].apply(parse_col)


df_fact_check_post_mapping = pd.read_csv(fact_check_post_mapping_path) 

In [4]:
# import csv

# # Der Pfad zur CSV-Datei
# csv_datei = posts_path

# # Öffnen der CSV-Datei und auslesen der 'verdicts' Spalte
# with open(csv_datei, newline='', encoding='utf-8') as csvfile:
#     # CSV-Reader erstellen
#     reader = csv.DictReader(csvfile)
    
#     # Set für einzigartige verdicts-Werte
#     verdicts_set = set()

#     # Iteriere durch jede Zeile in der CSV-Datei
#     for row in reader:
#         # Füge den Wert der 'verdicts'-Spalte zum Set hinzu
#         verdicts_set.add(row['verdicts'])

#     # Ausgabe der einzigartigen 'verdicts'-Werte
#     print("Mögliche 'verdicts'-Werte:")
#     for verdict in sorted(verdicts_set):
#         print(verdict)


In [5]:
# import pandas as pd

# # Load the CSV files
# file1 = posts_path
# file2 = fact_checks_path
# mapping_file = fact_check_post_mapping_path

# df1 = pd.read_csv(file1)
# df2 = pd.read_csv(file2)
# mapping = pd.read_csv(mapping_file)

# # Debugging: Check column names
# print("Columns in df1:", df1.columns)
# print("Columns in df2:", df2.columns)
# print("Columns in mapping:", mapping.columns)

# # Map IDs in df1 using the mapping file
# df1 = df1.merge(mapping, on="post_id", how="left")
# df1["mapped_id"] = df1["fact_check_id"]  # Add a new column for mapped IDs
# df1 = df1.drop(columns=["fact_check_id"])  # Drop unnecessary columns

# # No mapping needed for df2 as it already uses fact_check_id
# df2["mapped_id"] = df2["fact_check_id"]  # Rename for consistency

# # Merge the two DataFrames using the mapped ID column
# merged_df = pd.merge(df1, df2, on="mapped_id", how="inner")

# # Save the result to a new CSV file
# output_file = "merged_output.csv"
# merged_df.to_csv(output_file, index=False)

# print(f"Files merged successfully. Output saved to {output_file}")


In [ ]:
# df = pd.read_csv("merged_output.csv")
# output_file = "merged_output.csv"
# # Specify columns to delete
# columns_to_delete = ["fact_check_id"]  # Replace with actual column names ("instances_x", "text", "mapped_id", "instances_y",)

# # Delete the columns
# df = df.drop(columns=columns_to_delete)

# # Save the updated file
# df.to_csv(output_file, index=False)

# print(f"Updated CSV file saved to {output_file}")


Updated CSV file saved to merged_output.csv


In [10]:
import pandas as pd

# Load the CSV file
df = pd.read_csv("merged_output.csv")

# Set Pandas options to display full content without truncation
pd.set_option("display.max_colwidth", None)  # No truncation for column width
pd.set_option("display.max_rows", None)  # Show all rows (use with caution for large datasets)
pd.set_option("display.max_columns", None)  # Show all columns
pd.set_option("display.width", None)  # Automatically adjust width

# Print the first row of every column
for p in df.iloc[0].values:
    print(p)  # Access the first row (index 0)


0
[('! Dreister Impf-Fake von Markus Söder! Es ist wirklich unglaublich, wie dreist Spitzenpolitiker wie beispielsweise Markus Söder uns verarschen. Auf Instagram macht Söder fleißig Werbung für das Impfen Doch wenn man genau hinschaut, sieht man, dass er sich gar nichts injizieren lässt. Der Deckel ist nämlich noch auf der Nadel. Da könnt ihr mal sehen, wie sehr diejenigen, die euch zwangsimpfen wollen, den Impfstoffen vertrauen! markus.soeder FSME ...', "! Brazen vaccination fake by Markus Söder! It's really unbelievable how bold Top politicians such as Markus Söder kidding us. On Instagram does Söder busy advertising for vaccination But if you look closely, you can see you that he can't be injected at all. The lid is still on the needle. You can see how much those who want to vaccinate you, the Trust vaccines! markus.soeder TBE ...", [('deu', 0.9577142000198364), ('fy', 0.03401247784495354)])]
['False information']
('Markus Söder hat seine Impfung vorgetäuscht.', 'Markus Söder faked

In [3]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Hyperparameter
batch_size = 64
num_epochs = 10
learning_rate = 0.001
input_size = 300  # Größe des Word Embeddings (z.B. GloVe 300d)
hidden_size = 512  # Größe der versteckten Schicht
max_length = 50  # Maximale Länge der Eingabesequenz (Nummer der Wörter)

# Beispiel für die Word Embedding Initialisierung (z.B. GloVe)
embedding_matrix = np.random.rand(10000, input_size)  # 10.000 Vokabulargröße, Embedding-Größe 300

class FactCheckDataset(Dataset):
    def __init__(self, posts, claims, labels, tokenizer, max_length, embedding_matrix):
        self.posts = posts
        self.claims = claims
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.embedding_matrix = embedding_matrix

    def __len__(self):
        return len(self.posts)

    def __getitem__(self, idx):
        post = self.posts[idx]
        claim = self.claims[idx]
        label = self.labels[idx]
        
        # Tokenisierung und Indexierung der Wörter
        post_indices = self.tokenizer(post)
        claim_indices = self.tokenizer(claim)
        
        # Padding und Truncation
        post_indices = post_indices[:self.max_length] + [0] * (self.max_length - len(post_indices))
        claim_indices = claim_indices[:self.max_length] + [0] * (self.max_length - len(claim_indices))
        
        # Konvertiere Indizes zu Embeddings
        post_embedding = torch.tensor([self.embedding_matrix[idx] for idx in post_indices], dtype=torch.float32)
        claim_embedding = torch.tensor([self.embedding_matrix[idx] for idx in claim_indices], dtype=torch.float32)
        
        # Kombiniere Post und Claim-Embeddings
        combined_embedding = torch.cat((post_embedding, claim_embedding), dim=0)  # Verkettung der beiden

        return {
            'embedding': combined_embedding,
            'label': torch.tensor(label, dtype=torch.long)
        }

class FactCheckModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(FactCheckModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)  # Fully connected layer
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)  # Output layer (2 Klassen: relevant, nicht relevant)

    def forward(self, x):
        x = self.fc1(x)  # Erste Schicht
        x = self.relu(x)  # ReLU Aktivierung
        x = self.fc2(x)  # Ausgabeschicht
        return x

# Simulierter Tokenizer: wandelt Wörter in Indizes um (in der Realität verwendest du einen echten Tokenizer)
def simple_tokenizer(text):
    # Dummy-Tokenizer: Splitte Text in Wörter und ordne jedem Wort eine ID zu
    word_to_index = {"post": 1, "claim": 2, "fact": 3, "relevant": 4, "irrelevant": 5}  # Beispielwortschatz
    return [word_to_index.get(word, 0) for word in text.lower().split()]

# Beispiel-Daten
posts = ["post text 1", "post text 2"]
claims = ["claim text 1", "claim text 2"]
labels = [1, 0]  # 1: relevant, 0: nicht relevant

# Dataset erstellen
dataset = FactCheckDataset(posts, claims, labels, simple_tokenizer, max_length, embedding_matrix)
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Modell initialisieren
model = FactCheckModel(input_size=2*max_length*input_size, hidden_size=hidden_size, num_classes=2)

# Optimierer und Verlustfunktion
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# Training
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for batch in train_loader:
        optimizer.zero_grad()
        
        embeddings = batch['embedding']
        labels = batch['label']
        
        # Forward Pass
        outputs = model(embeddings)
        
        # Verlust berechnen
        loss = criterion(outputs, labels)
        
        # Backward Pass und Optimierung
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

# Funktion zur Berechnung der Top 10 relevantesten Behauptungen für einen gegebenen Post
def get_top_k_relevant_claims(post, claims, k=10):
    # Tokenisiere den Post
    post_indices = simple_tokenizer(post)
    post_embedding = torch.tensor([embedding_matrix[idx] for idx in post_indices], dtype=torch.float32)
    
    # Berechne die Ähnlichkeit zwischen dem Post und jeder Behauptung
    similarities = []
    for claim in claims:
        # Tokenisiere die Behauptung
        claim_indices = simple_tokenizer(claim)
        claim_embedding = torch.tensor([embedding_matrix[idx] for idx in claim_indices], dtype=torch.float32)
        
        # Berechne die Ähnlichkeit (z.B. Cosine Similarity)
        similarity = cosine_similarity(post_embedding.unsqueeze(0), claim_embedding.unsqueeze(0))
        similarities.append(similarity[0][0])
    
    # Sortiere die Behauptungen nach der Ähnlichkeit und gib die Top-k zurück
    top_k_indices = np.argsort(similarities)[-k:][::-1]  # Top-k Indizes, absteigend sortiert
    return [(claims[idx], similarities[idx]) for idx in top_k_indices]

# Beispiel: Top 10 relevante Behauptungen für einen Post
post_example = "post text 1"
top_10_claims = get_top_k_relevant_claims(post_example, claims, k=2)

print("Top 10 relevante Behauptungen:")
for claim, similarity in top_10_claims:
    print(f"Behauptung: {claim} - Ähnlichkeit: {similarity:.4f}")


c:\Users\schmi\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\schmi\AppData\Local\Temp\ipykernel_15252\3216602316.py:44: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:277.)
  post_embedding = torch.tensor([self.embedding_matrix[idx] for idx in post_indices], dtype=torch.float32)


RuntimeError: mat1 and mat2 shapes cannot be multiplied (200x300 and 30000x512)